# Libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

import cv2
import os
from tqdm import tqdm
from random import shuffle
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

# Data

In [8]:
train_dir = r'.\data\training_set'
test_dir= r'.\data\test_set'

# dataset labelling

In [9]:
def labeling(img):
    if img.split('.')[0]=='cat': return [0,1]
    else: return [1, 0]

# training data

In [10]:
class_names=['cats','dogs']
def create_train_data():
    training_data=[]
    #r"{}/scalers/{}".format(directory,file_name)
    for name in class_names:
        for img in tqdm(os.listdir(r"{}\{}".format(train_dir,name))):
            label=labeling(img)
            path=os.path.join(train_dir,name,img)
            #print(path)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
            #print(img.shape)
            img = cv2.resize(img, (50, 50))
            training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    #np.save('train_data.npy', training_data) 
    return training_data 

In [11]:
training_data=create_train_data()

100%|█████████████████████████████████████████████████████████████████████████████| 4005/4005 [00:25<00:00, 157.46it/s]


In [12]:
def create_test_data():
    test_data=[]
    #r"{}/scalers/{}".format(directory,file_name)
    for name in class_names:
        for img in tqdm(os.listdir(r"{}\{}".format(test_dir,name))):
            label=labeling(img)
            path=os.path.join(test_dir,name,img)
            #print(path)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
            #print(img.shape)
            img = cv2.resize(img, (50, 50))
            test_data.append([np.array(img), np.array(label)])
            
    shuffle(test_data)
    #np.save('train_data.npy', training_data) 
    return test_data 

In [13]:
test_data=create_test_data()

100%|█████████████████████████████████████████████████████████████████████████████| 1012/1012 [00:06<00:00, 166.21it/s]


# model

In [71]:
def create_model(input_shape,num_classes):

    
    model=Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='softmax'))

    return model
    

In [72]:
# Define the input shape of your images and the number of classes
input_shape = (50, 50, 1)  # e.g., (64, 64, 3)
num_classes = 2  # Number of classes for classification

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [73]:
X = np.array([i[0] for i in training_data]).reshape(-1, 50, 50, 1) 
Y = np.array([i[1] for i in training_data]) 

In [74]:
training_data[2][1]

array([0, 1])

In [75]:
Y.shape

(8005, 2)

In [76]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)                   │ (None, 48, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 48, 48, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 22, 22, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 22, 22, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 11, 11, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 11, 11, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 9, 9, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 9, 9, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,145,730 (4.37 MB)

 Trainable params: 1,144,258 (4.36 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [77]:
# simple early stopping
es = EarlyStopping(monitor='val_loss', verbose=2, patience=5)

In [78]:
# Train the model
model.fit(X, Y, epochs=50, batch_size=512, validation_split=0.2,callbacks=[es])

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 533ms/step - accuracy: 0.5579 - loss: 1.0142 - val_accuracy: 0.5003 - val_loss: 2.1321
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 513ms/step - accuracy: 0.6307 - loss: 0.7852 - val_accuracy: 0.5072 - val_loss: 1.9087
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 514ms/step - accuracy: 0.6622 - loss: 0.7303 - val_accuracy: 0.6465 - val_loss: 0.7695
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 512ms/step - accuracy: 0.6913 - loss: 0.6619 - val_accuracy: 0.6334 - val_loss: 0.6771
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 514ms/step - accuracy: 0.7095 - loss: 0.6249 - val_accuracy: 0.6640 - val_loss: 0.6859
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 513ms/step - accuracy: 0.7308 - loss: 0.5833 - val_accuracy: 0.6921 - val_loss: 0.6021
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 516ms/step - accuracy: 0.7478 - loss: 0.5619 - val_accuracy: 0.6640 - val_loss: 0.6408
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 515ms/step - accuracy: 0.7523 - loss: 0.5478 - val_accuracy: 0

# predictions

In [79]:
x_test = np.array([i[0] for i in test_data]).reshape(-1, 50, 50, 1) 
y_test = np.array([i[1] for i in test_data]) 

In [80]:
 #Predict labels for images
def predict_images(model, images):
    predictions = model.predict(images)
    return predictions

# Interpret predictions
def interpret_predictions(predictions):
    # For each prediction, print the class with the highest probability
    for i, pred in enumerate(predictions):
        max_prob_index = np.argmax(pred)
        print(f"Image {i+1}: Class {max_prob_index}, Probability {pred[max_prob_index]}")

# Paths to the images you want to predict
image_paths = ['path_to_image1.jpg', 'path_to_image2.jpg', ...]  # Add your image paths here



# Predict labels for the images
predictions = predict_images(model, x_test)

# Interpret predictions
y_pred=interpret_predictions(predictions)

64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Image 1: Class 1, Probability 0.9087627530097961
Image 2: Class 1, Probability 0.9996298551559448
Image 3: Class 1, Probability 0.9998173117637634
Image 4: Class 0, Probability 0.9778889417648315
Image 5: Class 1, Probability 0.9992545247077942
Image 6: Class 1, Probability 0.9967249035835266
Image 7: Class 0, Probability 0.8524469137191772
Image 8: Class 1, Probability 0.9999657869338989
Image 9: Class 1, Probability 0.9997542500495911
Image 10: Class 0, Probability 0.8948307633399963
Image 11: Class 1, Probability 0.9999368190765381
Image 12: Class 1, Probability 0.9991613626480103
Image 13: Class 0, Probability 0.8055426478385925
Image 14: Class 1, Probability 0.9996116757392883
Image 15: Class 1, Probability 1.0
Image 16: Class 0, Probability 0.9397193789482117
Image 17: Class 0, Probability 0.8843091726303101
Image 18: Class 1, Probability 0.8072633743286133
Image 19: Class 1, Probability 0.9687400460243225
Image 20: Class 1, Probability 0.9

In [81]:
# Convert predictions to binary labels (0 or 1)
predicted_labels = (predictions > 0.5).astype(np.int32)

# Calculate accuracy
accuracy = np.mean(predicted_labels == y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7889273356401384
